In [0]:
!pip install ipywidgets

In [0]:
%matplotlib inline
import os
from io import BytesIO

import ipywidgets as ipy
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
from scipy.stats import zscore
from skimage import io
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.preprocessing import minmax_scale
from sklearn.manifold import TSNE

from google.colab import files

## 1. read single image, and display it

#### read image

In [0]:
uploaded = files.upload()

In [0]:
img = uploaded['timepoint_12.tiff']
single_img = io.imread(BytesIO(img))

#### check data type and shape

In [0]:
print(type(single_img))
print(single_img.shape)

#### plot single image

In [0]:
fig, ax = plt.subplots(figsize=(4, 8)) # here you can change figure size
ax.matshow(single_img[28,:,:], cmap='hot') # integer number is layer index (here 28)

# Load dataset

We will need data in two different forms
1. `images_dict`. It it dictionary (Python data structure)
2. `images_matrix`. 2-dimensional numpy array

Select all timepoints, that is 100 tiff files

In [0]:
uploaded = files.upload()

In [0]:
images_dict = dict()
images_matrix = np.zeros([100, 40*256*180])

for i in range(100):
  img_name = 'timepoint_{:02d}.tiff'.format(i)
  img_bytes = uploaded[img_name]
  img = io.imread(BytesIO(img_bytes))
  images_dict[img_name[:-5]] = img
  images_matrix[i] = np.reshape(img, [-1], order='C')
print(images_matrix.shape)

# Work towards PCA analysis

In order to perform PCA analysis, images need to be preprocessed in the certain way. This part of the notebook will guide you through this process.

## 2. subtract background

#### interactive widget
Here you can interactively explore which threshold value is most appropriate.
1. threshold slider -> to pick different values of the threshold
2. img_layer slider -> to browse different layer in the images stack

In [0]:
@ipy.interact(threshold=ipy.widgets.IntSlider(value=90, min=90, max=170))
def f(threshold):
    timepoint = np.copy(images_dict['timepoint_12'])
    timepoint[timepoint <= threshold] = 0

    @ipy.interact(img_layer=ipy.widgets.IntSlider(value=26, min=0, max=39))
    def f(img_layer):
        fig, ax = plt.subplots(figsize=(5,10))
        ax.matshow(timepoint[img_layer,:,:], cmap='hot')

#### non-interactive version

In [0]:
threshold = 120 # manipulate this
timepoint = np.copy(images_dict['timepoint_12'])
timepoint[timepoint <= threshold] = 0
fig, ax = plt.subplots(figsize=(5,10))
ax.matshow(timepoint[26,:,:], cmap='hot') # review several image layers

### 2.1 compare before-after

In [0]:
@ipy.interact(img_layer=ipy.widgets.IntSlider(value=26, min=0, max=39))
def f(img_layer):
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10,10))
    ax1.matshow(images_dict['timepoint_12'][img_layer,:,:], cmap='hot')
    ax1.set_title('before', size='26')
    ax2.matshow(timepoint[img_layer,:,:], cmap='hot')
    ax2.set_title('after', size='26')

In [0]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10,10))
ax1.matshow(images_dict['timepoint_12'][26,:,:], cmap='hot')
ax1.set_title('before', size='26')

ax2.matshow(timepoint[26,:,:], cmap='hot')
ax2.set_title('after', size='26')

## 3. remove pixels with zeros

### 3.1 get indices with nonzero pixels

Procedure is as follows:
1. vectorize image stack `timepoint`.
2. get only nonzero positions in the vector

As a result the `inx` stores indices of nonzero values.

In [0]:
timepoint_vector = np.reshape(timepoint, [-1], order='C')
timepoint_vector.shape

inx = np.nonzero(timepoint_vector)
print(inx[0])
print(len(inx[0]))

### 3.2 get only foreground pixels

In [0]:
images_matrix_fg = np.zeros([100, len(inx[0])]) # fg = foreground
for i in range(images_matrix.shape[0]):
    images_matrix_fg[i,:] = images_matrix[i,inx[0]]

print('images matrix has shape:            {}'.format(images_matrix.shape))
print('images matrix foreground has shape: {}'.format(images_matrix_fg.shape))

## 4. normalize input

In [0]:
images_matrix_fg_norm = zscore(images_matrix_fg, axis=1)
images_matrix_fg_norm = np.nan_to_num(images_matrix_fg_norm)

#im2d_n = zscore(im2d_fg, axis=0) # im2d_n = image 2d normalized (double check the axis (0,1,None))
#im2d_n = np.nan_to_num(im2d_n)

### 4.1 check normalization

In [0]:
print('max value:   {}'.format(images_matrix_fg_norm.max()))
print('min value:   {}'.format(images_matrix_fg_norm.min()))
print('mean values: {}'.format(images_matrix_fg_norm.mean(axis=1)))
print('std values:  {}'.format(images_matrix_fg_norm.std(axis=1)))
#im2d_n.std(axis=0)

## 5. clip outliers

In [0]:
images_matrix_fg_norm_clipped = np.clip(images_matrix_fg_norm, a_min=-3.0, a_max=3.0)
print('min: {}'.format(images_matrix_fg_norm_clipped.min()))
print('max: {}'.format(images_matrix_fg_norm_clipped.max()))

## 6. calculate PCA

In [0]:
n_components = 50
pca = PCA(n_components=n_components)
pca_space = pca.fit_transform(images_matrix_fg_norm_clipped)

### 6.1 check output

In [0]:
print('output data type: ' + str(type(pca_space)))
print('output shape:     ' + str(pca_space.shape))

#### 6.1 prepare principal components for visualization
Reshape back to the original size (256, 180, 40)

In [0]:
pca_space_bg = np.zeros([40*256*180, n_components]) # pca_space_bg = PCA space with background (zeros)
for i in range(pca_space_bg.shape[1]):
    pca_space_bg[inx, i]= pca.components_[i,:] # Note that here we make use of the previously computed inx
pca_space_bg = pca_space_bg.reshape([40, 256, 180, -1])
print(pca_space_bg.shape)

## 7. visualize principal components

#### interactive widget

In [0]:
@ipy.interact(principal_component=ipy.widgets.IntSlider(value=0, min=0, max=59))
def f(principal_component):
    @ipy.interact(img_layer=ipy.widgets.IntSlider(value=26, min=0, max=39))
    def f(img_layer):
        fig, ax = plt.subplots(figsize=(7,7))
        cax = ax.matshow(pca_space_bg[img_layer,:,:, principal_component], cmap='PiYG')
        fig.colorbar(cax)

#### Interactive wigdet: display only max projections (easier to visually inspect)

In [0]:
pca_abs = np.abs(pca_space_bg)
@ipy.interact(principal_component=ipy.widgets.IntSlider(value=0, min=0, max=59))
def f(principal_component):
    fig, ax = plt.subplots(figsize=(7,7))
    ax.matshow(pca_abs.max(axis=0)[:,:,principal_component], cmap='hot')

#### non-interactive: display only max projections (easier to visually inspect)

In [0]:
pca_abs = np.abs(pca_space_bg)

In [0]:
fig, ax = plt.subplots(figsize=(7,7))
ax.matshow(pca_abs.max(axis=0)[:,:,0], cmap='hot') # change int here

In [0]:
# three plot on one figure
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(12,12))
ax1.matshow(pca_abs.max(axis=0)[:,:,0], cmap='hot') # change int here (range = 0, n_components)
ax2.matshow(pca_abs.max(axis=0)[:,:,1], cmap='hot') # change int here (range = 0, n_components)
ax3.matshow(pca_abs.max(axis=0)[:,:,2], cmap='hot') # change int here (range = 0, n_components)

## 12. visualize variance explained ratio

In [0]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10,5), facecolor='w')
ax1.plot(pca.explained_variance_ratio_, color='black', lw=2)
ax1.grid(axis='both')
ax1.set_ylabel('proportion of variance explained', fontsize=10)
ax1.set_xlabel('principal component', fontsize=10)

ax2.plot(np.cumsum(pca.explained_variance_ratio_), color='black', lw=2)
ax2.grid(axis='both')
ax2.set_ylabel('cumulative proportion of variance explained', fontsize=10)
ax2.set_xlabel('number of components', fontsize=10)

### end of the PCA analysis

If this task was easy for you, you may want to try different dimensionality reduction methods. Sklearn offers coherent API for many methods.
Please refer to sklearn.decomposition module: http://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition.

# calculate t-SNE embeddings on the PCA space

In [0]:
from sklearn.manifold import TSNE

### prepare PCA space

In [0]:
n_components = 99
pca = PCA(n_components=n_components)
pca_space = pca.fit_transform(images_matrix_fg_norm_clipped)
print('pca space dimensions: {}'.format(pca_space.shape))

### explore t-SNE embeddings as a function of the perplexity

In [0]:
@ipy.interact(perplexity=ipy.widgets.IntSlider(value=29, min=0, max=50))
def f(perplexity):
    tsne = TSNE(n_components=2, n_iter=1000,  perplexity=perplexity)
    tsne_output = tsne.fit_transform(pca_space)
    print('t-SNE output has shape: {}'.format(tsne_output.shape))
    fig, ax = plt.subplots(figsize=(7,7))
    ax.scatter(tsne_output[:,0], tsne_output[:,1])
    ax.grid(axis='both')

# cluster t-SNE embeddings using DBSCAN

In [0]:
from sklearn.cluster import DBSCAN

### explore clustering results as a function of two parameters: eps and nim_samples

In [0]:
tsne = TSNE(n_components=2, n_iter=1000,  perplexity=29)
tsne_output = tsne.fit_transform(pca_space)
@ipy.interact(eps=ipy.widgets.FloatSlider(value=1.2, min=0, max=10))
def f(eps):
    @ipy.interact(min_samples=ipy.widgets.IntSlider(value=5, min=3, max=50))
    def g(min_samples):
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        dbscan_output = dbscan.fit_predict(tsne_output)
        plt.scatter(x=tsne_output[:,0], y=tsne_output[:,1], c=dbscan_output)
        plt.grid()
        print(dbscan_output)

# Final remarks

In [0]:
final_chart = pca_abs.max(axis=0)[:,:,[2,9,12]]
final_chart = final_chart / final_chart.max()
final_chart = final_chart*255
img = Image.fromarray(final_chart.astype(np.uint8))

fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(img)

# end